In [1]:
import sys
sys.path.append('..')

In [2]:
import torch
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import random
import torch.nn as nn
import torch.nn.functional as F

In [3]:
ligands = ["TNF", "R84", "PIC", "P3K", "FLA", "CpG", "FSL", "LPS", "UST"]
polarization = ["", "ib", "ig", "i0", "i3", "i4"]
replicas, size = 1, 1288 # unreplicated

In [4]:
from core.getdata import *
from core.dataset import *

<h3>Example of GetData</h3>

In [5]:
TNFib1 = GetData(ligands[0], polarization[1], replicas, size)
TNFib1.X.shape

(1288, 98)

<h3>Example of Dataset</h3>

In [6]:
data = Dataset(ligands, polarization, replicas, size)

In [7]:
%%time
for _ in data:
    pass

Wall time: 40 ms


In [8]:
data.data.shape

(69552, 98, 1)

In [9]:
print(type(data.data), type(data.labels))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [10]:
print(data.labels, data.labels.shape) #labels are multi-hot encoded

[0 0 0 ... 8 8 8] (69552,)


<h3>Initializing Dataloaders</h3>

In [11]:
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [12]:
X_len, test_len = int(len(data.data) * 0.9), int(len(data.data) * 0.1)
train_len, val_len = int(X_len * 0.9), int(X_len * 0.1)
print(X_len, test_len, train_len, val_len) #lengths
print(X_len + test_len, len(data.data))

62596 6955 56336 6259
69551 69552


In [13]:
training_data = data.data.reshape(data.data.shape[0], data.data.shape[1], 1) #adds extra dimension

In [14]:
data.data[1000].shape
len(data)

69552

In [15]:
dataset_X, dataset_val = torch.utils.data.random_split(data, [X_len, test_len + 1]) # need separate data and labels for LSTM?

In [16]:
dataloader_train = torch.utils.data.DataLoader(dataset_X, batch_size=64, shuffle=True)
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=64, shuffle=True)

In [17]:
print(len(dataloader_train), len(dataloader_val), len(dataloader_train) + len(dataloader_val))
###
#data[0:10]

979 109 1088


<h3>Model Training</h3>

In [18]:
from core.network import *
from core.trainer import *

In [19]:
torch.cuda.is_available()

True

In [20]:
#model parameters
input_size = 1
hidden_sizes = 98
output_size = 9

#training parameters
n_epochs = 80
batch_size = 64
learning_rate = 1e-3

In [21]:
model = LSTM(input_size, hidden_sizes, output_size, num_layers=3, device="cuda:0")
model.train()

LSTM(
  (lstm): LSTM(1, 98, num_layers=3, batch_first=True)
  (fc1): Linear(in_features=98, out_features=9, bias=True)
)

In [22]:
trainer = LSTMTrainer(model=model, device="cuda:0")

In [ ]:
trainer.train(dataloader_train, dataloader_val, batch_size=batch_size, n_epochs=n_epochs)

  1%|█                                                                                  | 1/80 [00:14<19:19, 14.68s/it]

Epoch 001: | Training Loss: 2.1598007090853963 | Validation Loss: 2.1578694308569673


  2%|██                                                                                 | 2/80 [00:28<18:13, 14.02s/it]

Epoch 002: | Training Loss: 1.9780745944645601 | Validation Loss: 1.8644620158256742


  4%|███                                                                                | 3/80 [00:42<18:06, 14.11s/it]

...

Epoch 040: | Training Loss: 0.8855489613269517 | Validation Loss: 1.0186585220721884


 51%|██████████████████████████████████████████                                        | 41/80 [11:26<10:55, 16.80s/it]

Epoch 041: | Training Loss: 0.8508554427019788 | Validation Loss: 1.0019755554855416


 52%|███████████████████████████████████████████                                       | 42/80 [11:43<10:35, 16.72s/it]

Epoch 042: | Training Loss: 0.8269959952419698 | Validation Loss: 0.9957441851633404


<h3>Evaluation</h3>

In [ ]:
torch.cuda.empty_cache()
x_batch, y_batch = dataset_val[0:10000]
# to do : convert x_batch to tensor and send to gpu 
x_batch = torch.tensor(x_batch, device=torch.device("cuda:0"))
y_pred = trainer.network(x_batch)
y_pred = F.softmax(y_pred, dim=1)
# to do : convert to np array and vstack it to y_pred
y_pred = y_pred.detach().cpu().numpy()
y_pred = np.argmax(y_pred, axis=1)
dic = {"y_pred": y_pred, "y_true": y_batch}

df = pd.DataFrame(dic)

In [ ]:
df

In [ ]:
print(f' Accuracy: {sum(df["y_pred"] == df["y_true"])/6956}')

In [ ]:
dir_save = '../models/'
trainer.save(dir_save + 'lstm2.pth')# model trained on replicated dataset

<h3>Plots</h3>

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(trainer.val_losses)
plt.plot(trainer.train_losses)

Overfitting seems to occur ~50 epochs | Overall accuracy of the model seems to be the same across replicated/unreplicated datasets